In [1]:
#!mongoimport --type csv -d Project_3 -c video_game_sales --drop --jsonArray /Data/SQL Video_Game_Sales.csv
#!mongoimport --type csv -d Project_3 -c video_games --drop --jsonArray /Data/SQL video_games_rating.csv


In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from pprint import pprint 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
mongo = MongoClient(port=27017)

In [4]:
print(mongo.list_database_names())

['Homework1', 'Project_3', 'admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [5]:
db = mongo['Project_3']

In [6]:
print(db.list_collection_names())

['video_games', 'video_game_sales']


In [7]:
video_games = db['video_games']

In [8]:
pprint(db.video_games.find_one())

{'_id': ObjectId('6584b53934d390600f49621d'),
 'name': 'The Legend of Zelda: Ocarina of Time',
 'platform': ' Nintendo 64',
 'release_date': '23-Nov-98',
 'summary': 'As a young boy, Link is tricked by Ganondorf, the King of the '
            'Gerudo Thieves. The evil human uses Link to gain access to the '
            'Sacred Realm, where he places his tainted hands on Triforce and '
            'transforms the beautiful Hyrulean landscape into a barren '
            'wasteland. Link is determined to fix the problems he helped to '
            'create, so with the help of Rauru he travels through time '
            'gathering the powers of the Seven Sages.',
 'user_review': 9.1}


In [9]:
count_tbd = video_games.count_documents({'user_review': 'tbd'})

print(f"Number of documents where user_review is 'tbd': {count_tbd}")

Number of documents where user_review is 'tbd': 0


In [10]:
# Delete documents with user_review equal to "tbd"
result = video_games.delete_many({'user_review': 'tbd'})
print(f"{result.deleted_count} documents deleted")
# Now, retrieve the remaining documents and perform aggregation
pipeline = [
    {
        '$group': {
            '_id': '$name',
            'mean_user_review': {'$avg': '$user_review'}
        }
    }
]

result_after_deletion = list(video_games.aggregate(pipeline))

for item in result_after_deletion[:5]:
    pprint(item)

0 documents deleted
{'_id': 'Dynasty Warriors 8: Xtreme Legends - Definitive Edition',
 'mean_user_review': 8.5}
{'_id': 'Anno 1800', 'mean_user_review': 7.9}
{'_id': 'Galactic Civilizations II: Dark Avatar', 'mean_user_review': 7.8}
{'_id': 'Delta Force - Black Hawk Down: Team Sabre', 'mean_user_review': 7.4}
{'_id': 'Dark Souls II: Crown of the Old Iron King', 'mean_user_review': 7.85}


In [11]:
documents_with_tbd = video_games.find({'user_review': 'tbd'})

# Print or process the matching documents
for document in documents_with_tbd:
    print(document)

In [12]:
cleaned_df = pd.DataFrame(result_after_deletion)
cleaned_df.head(15)

,_id,mean_user_review
0,Dynasty Warriors 8: Xtreme Legends - Definitiv...,8.50
1,Anno 1800,7.90
2,Galactic Civilizations II: Dark Avatar,7.80
3,Delta Force - Black Hawk Down: Team Sabre,7.40
4,Dark Souls II: Crown of the Old Iron King,7.85
5,Strong Bad's Cool Game for Attractive People E...,7.00
6,King's Quest Chapter 5: The Good Knight,5.50
7,Pryzm Chapter One: The Dark Unicorn,3.60
8,Hearthstone: Heroes of Warcraft - Blackrock Mo...,5.80
9,Battletoads,5.85


In [13]:
cleaned_df['mean_user_review'] = cleaned_df['mean_user_review'].round(2)
cleaned_df = cleaned_df.rename(columns={'_id': 'Video Game'})
cleaned_df = cleaned_df.rename(columns={'mean_user_review': 'Average Rating'})

cleaned_df.head(15)

,Video Game,Average Rating
0,Dynasty Warriors 8: Xtreme Legends - Definitiv...,8.50
1,Anno 1800,7.90
2,Galactic Civilizations II: Dark Avatar,7.80
3,Delta Force - Black Hawk Down: Team Sabre,7.40
4,Dark Souls II: Crown of the Old Iron King,7.85
5,Strong Bad's Cool Game for Attractive People E...,7.00
6,King's Quest Chapter 5: The Good Knight,5.50
7,Pryzm Chapter One: The Dark Unicorn,3.60
8,Hearthstone: Heroes of Warcraft - Blackrock Mo...,5.80
9,Battletoads,5.85


In [14]:
top_games = cleaned_df.sort_values(by='Average Rating', ascending=False)
top_games.head(15)

,Video Game,Average Rating
2962,Metal Torrent,9.70
7258,Tengami,9.70
4393,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,9.70
6818,Ghost Trick: Phantom Detective,9.70
908,GrimGrimoire,9.70
10789,Crystar,9.60
2605,Diaries of a Spaceport Janitor,9.60
5105,Astro's Playroom,9.30
6610,The Witcher 3: Wild Hunt,9.27
2873,The Witcher 3: Wild Hunt - Blood and Wine,9.20


In [15]:
bottom_games = cleaned_df.sort_values(by='Average Rating', ascending=True)
bottom_games.head(15)

,Video Game,Average Rating
7251,Madden NFL 21,0.35
4939,Madden NFL 22,0.55
6071,Warcraft III: Reforged,0.60
7901,FIFA 20: Legacy Edition,0.70
2150,When Ski Lifts Go Wrong,1.00
6464,The Sims 4: Star Wars - Journey to Batuu,1.00
4524,FIFA 21,1.07
4877,Call of Duty: Modern Warfare 3 - Defiance,1.20
9659,FIFA 20,1.30
2170,Fast & Furious: Showdown,1.30


In [16]:
print(db.list_collection_names())

['video_games', 'video_game_sales']


In [17]:
video_game_sales = db['video_game_sales']

In [18]:
pprint(db.video_game_sales.find_one())

{'EU_Sales': 29.02,
 'Genre': 'Sports',
 'Global_Sales': 82.74,
 'JP_Sales': 3.77,
 'NA_Sales': 41.49,
 'Name': 'Wii Sports',
 'Other_Sales': 8.46,
 'Platform': 'Wii',
 'Publisher': 'Nintendo',
 'Rank': 1,
 'Year': 2006,
 '_id': ObjectId('6584b56634d390600f49ab8d')}


In [19]:
result2 = video_game_sales.delete_many({'Year': 'N/A'})
print(f"{result2.deleted_count} documents deleted")
pipeline = [
    {
        '$group': {
            '_id': '$Name',
            'Total Global Sales': {'$sum': '$Global_Sales'}
        }
    }
]

global_sales = list(video_game_sales.aggregate(pipeline))

for item in global_sales[:5]:
    pprint(item)

271 documents deleted
{'Total Global Sales': 0.14, '_id': 'Fear Factor: Unleashed'}
{'Total Global Sales': 0.04, '_id': 'Emergency Room: Real Life Rescues'}
{'Total Global Sales': 0.51, '_id': 'StarCraft II: Legacy of the Void'}
{'Total Global Sales': 2.8, '_id': 'WWE SmackDown! Here Comes the Pain'}
{'Total Global Sales': 0.05, '_id': 'Tour de France 2014'}


In [20]:
global_df = pd.DataFrame(global_sales)
cleaned_sales_df = global_df.rename(columns={'_id': 'Video Game'})
top_sales = cleaned_sales_df.sort_values(by='Total Global Sales', ascending=False)
top_sales.head(15)

,Video Game,Total Global Sales
8290,Wii Sports,82.74
1481,Grand Theft Auto V,55.92
518,Super Mario Bros.,45.31
10445,Tetris,35.84
8590,Mario Kart Wii,35.82
5619,Wii Sports Resort,33.00
3048,Pokemon Red/Pokemon Blue,31.37
6711,Call of Duty: Modern Warfare 3,30.83
7374,New Super Mario Bros.,30.01
3705,Call of Duty: Black Ops II,29.72


In [21]:
pipeline2 = [
    {
        '$group': {
            '_id': '$Genre',
            'Total Global Sales': {'$sum': '$Global_Sales'}
        }
    }
]

per_category = list(video_game_sales.aggregate(pipeline2))

for item in per_category[:5]:
    pprint(item)

{'Total Global Sales': 444.05, '_id': 'Fighting'}
{'Total Global Sales': 173.43, '_id': 'Strategy'}
{'Total Global Sales': 390.16, '_id': 'Simulation'}
{'Total Global Sales': 1026.2, '_id': 'Shooter'}
{'Total Global Sales': 242.22, '_id': 'Puzzle'}


In [22]:
global_df_per_catergory = pd.DataFrame(per_category)
cleaned_sales_df_category = global_df_per_catergory.rename(columns={'_id': 'Video Game'})
top_sales_per_category = cleaned_sales_df_category.sort_values(by='Total Global Sales', ascending=False)
top_sales_per_category.head(15)

,Video Game,Total Global Sales
5,Action,1722.88
8,Sports,1309.24
3,Shooter,1026.20
11,Role-Playing,923.84
9,Platform,829.15
6,Misc,797.62
10,Racing,726.77
0,Fighting,444.05
2,Simulation,390.16
4,Puzzle,242.22


In [23]:
pipeline3 = [
    {
        '$group': {
            '_id': '$Platform',
            'Total Global Sales': {'$sum': '$Global_Sales'}
        }
    }
]

per_platform = list(video_game_sales.aggregate(pipeline3))

for item in per_platform[:5]:
    pprint(item)

{'Total Global Sales': 818.96, '_id': 'DS'}
{'Total Global Sales': 218.21, '_id': 'N64'}
{'Total Global Sales': 200.05, '_id': 'SNES'}
{'Total Global Sales': 255.05, '_id': 'PC'}
{'Total Global Sales': 313.56, '_id': 'GBA'}


In [24]:
global_df_per_platform = pd.DataFrame(per_platform)
cleaned_sales_df_platform = global_df_per_platform.rename(columns={'_id': 'Platform'})
top_sales_per_platform = cleaned_sales_df_platform.sort_values(by='Total Global Sales', ascending=False)
top_sales_per_platform.head(15)

,Platform,Total Global Sales
20,PS2,1233.46
11,X360,969.61
8,PS3,949.35
18,Wii,909.81
0,DS,818.96
21,PS,727.39
4,GBA,313.56
9,PSP,291.71
28,PS4,278.10
3,PC,255.05


In [25]:
# Find the document with the maximum date
most_recent_document = video_game_sales.find_one(sort=[("Year", pymongo.DESCENDING)])

# Extract the most recent date from the document
most_recent_date = most_recent_document["Year"]

print("Most Recent Date:", most_recent_date)


Most Recent Date: 2020


In [26]:
merged_df = pd.merge(cleaned_df, cleaned_sales_df, on='Video Game')
# Print the merged DataFrame
merged_df.head(15)

,Video Game,Average Rating,Total Global Sales
0,Pryzm Chapter One: The Dark Unicorn,3.60,0.12
1,Dynasty Warriors 4: Xtreme Legends,8.30,0.86
2,Mobile Light Force 2,7.20,0.11
3,Ace Combat 5: The Unsung War,8.90,1.71
4,Mega Man X: Command Mission,8.60,0.30
5,Syphon Filter 3,8.60,0.52
6,Resident Evil: Survivor,6.10,0.71
7,Jade Cocoon 2,7.90,0.10
8,The Typing of the Dead,7.60,0.05
9,Far Cry,8.00,0.05


In [27]:
top_rating_merged = merged_df.sort_values(by='Average Rating', ascending=False)
top_rating_merged.head(15)

,Video Game,Average Rating,Total Global Sales
342,GrimGrimoire,9.70,0.05
2599,Ghost Trick: Phantom Detective,9.70,0.42
2515,The Witcher 3: Wild Hunt,9.27,5.64
1427,Planescape: Torment,9.20,0.04
2439,The Last of Us,9.20,10.54
599,Warcraft III: The Frozen Throne,9.20,1.54
2156,Xenogears,9.20,1.46
1521,Suikoden II,9.20,0.68
1733,Deus Ex,9.20,0.01
3739,Warcraft III: Reign of Chaos,9.20,0.13


In [28]:
top_sales_merged = merged_df.sort_values(by='Total Global Sales', ascending=False)
top_sales_merged.head(15)

,Video Game,Average Rating,Total Global Sales
2773,Wii Sports,8.10,82.74
1071,Grand Theft Auto V,8.14,55.92
4079,Tetris,7.50,35.84
3698,Mario Kart Wii,8.40,35.82
1626,Wii Sports Resort,8.20,33.00
2303,Call of Duty: Modern Warfare 3,2.92,30.83
3075,New Super Mario Bros.,8.50,30.01
1303,Call of Duty: Black Ops II,5.65,29.72
4115,Call of Duty: Black Ops,6.36,29.40
183,Wii Play,6.60,29.02


In [31]:
fig = px.scatter(top_sales_merged.head(50), x='Average Rating', y='Total Global Sales',
                 title='Video Game Rating vs Global Sales',
                 labels={'Average Rating': 'Average Rating', 'Total Global Sales': 'Total Global Sales (in millions)'},
                 hover_name='Video Game', size_max=60)
fig.update_layout(
    showlegend=False,  # Remove the legend
    margin=dict(l=0, r=0, t=30, b=0),  # Adjust margin
)
# Show the plot
fig.show()